In [ ]:
import pandas as pd
import glob

# https://data.humdata.org/dataset/social-connectedness-index
# Read all .tsv files and concatenate them into one DataFrame
'''
file_paths = sorted(glob.glob("zcta_zcta_shard*.tsv"))  # Ensure files are read in order
df = pd.concat([pd.read_csv(f, sep="\t", dtype={"user_loc": str, "fr_loc": str}) for f in file_paths], ignore_index=True)

# Check the first few rows
print(df.head())

territory_zip_codes = ['006', '007', '008', '009', '967', '969']

df = df[~df['user_loc'].str.startswith(tuple(territory_zip_codes)) & ~df['fr_loc'].str.startswith(tuple(territory_zip_codes))]
'''

  user_loc fr_loc  scaled_sci
0    00601  00601    47263819
1    00601  00602      480097
2    00601  00603      338333
3    00601  00610      191037
4    00601  00612      292680


In [ ]:
'''df.to_csv("filtered_zip_sci_data.csv", index=False)'''

From now on just read in cleaned version

In [ ]:
df = pd.read_csv("filtered_zip_sci_data.csv", sep="\t", dtype={"user_loc": str, "fr_loc": str})

In [ ]:
import numpy as np

df["distance"] = 1 / df["scaled_sci"]

distance_matrix = df.pivot(index="user_loc", columns="fr_loc", values="distance").fillna(np.inf)

dist_array = distance_matrix.values

In [13]:
import gc

del df
gc.collect()

81

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# hierarchical clustering
clustering = AgglomerativeClustering(n_clusters=50, metric="precomputed", linkage="average")
clusters = clustering.fit_predict(dist_array)

zip_clusters = pd.DataFrame({"zip_code": distance_matrix.index, "cluster": clusters})
print(zip_clusters.head())


  zip_code  cluster
0    01001       21
1    01002       21
2    01003       21
3    01005       21
4    01007       21


In [15]:
zip_clusters.to_csv("zip_clusters.csv", index=False)

In [ ]:
%reset -f #clears all objects out of memory